#  Projeto Deep Learning 

## Importações

In [2]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input
import subprocess
from tensorflow.keras.applications import EfficientNetB3



## Carregamento do CSV e Verificação de imagens

In [3]:
metadata_path = "metadata.csv" 
meta_data = pd.read_csv(metadata_path)

base_dir = "/Users/joaosantos/Documents/Mestrado Joao/2 semestre/Deep Learning/rare_specie" 

In [ ]:
def check_image_exists(image_file):
    for root, dirs, files in os.walk(base_dir):
        if image_file in files:
            return True
    return False

In [ ]:
missing_images = []
for index, row in meta_data.iterrows():
    image_path = row['file_path']  
    image_name = os.path.basename(image_path)  
    if not check_image_exists(image_name):
        missing_images.append(image_path)

print(f"Número de imagens em falta: {len(missing_images)}")

## Load Images
Carregar imagens visto que cada pasta é uma classe (familia) e dar one hot enconding nas classes , ou seja, 
X_data são imagens e y_data são rótulos/classes codificados


Resizing usando 224x224, pois medidas mais baixas fazem perder qualidade de imagem e consequentemente pioram a performance

In [ ]:
# Função para remover fundo da imagem
def remove_background(input_path, output_path):
    try:
        print(f"Removendo fundo da imagem: {input_path}")
        # Executa o rembg para remover o fundo
        result = subprocess.run(['rembg', 'i', input_path, output_path], check=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Fundo removido com sucesso de {input_path}. Salvo em {output_path}")
        else:
            print(f"Erro ao processar {input_path}: {result.stderr}")
    except subprocess.CalledProcessError as e:
        print(f"Erro ao processar {input_path}: {e}")

In [4]:
# Funcão para carregar imagens e dar one hot enconding
def load_local_images(base_dir, image_size=(299, 299)):
    X_data = []
    y_data = []

    # Obter os rótulos 
    labels = os.listdir(base_dir)
    labels = [label for label in labels if os.path.isdir(os.path.join(base_dir, label)) and not label.startswith('.')]
    label_map = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        class_dir = os.path.join(base_dir, label)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Erro:{img_path}")
                continue
            img = cv2.resize(img, image_size)
            X_data.append(img)
            y_data.append(label_map[label])

    X_data = np.array(X_data, dtype="float32")  
    y_data = np.array(y_data)

    # One-hot encoding
    num_classes = len(labels)
    y_data = to_categorical(y_data, num_classes=num_classes)

    print(f"Classes encontradas: {labels}")
    return X_data, y_data



In [5]:
#Carregar os dados
image_size = (299, 299)
X_data, y_data = load_local_images(base_dir, image_size=image_size)

# Primeiro split: separar o conjunto de teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

# Segundo split: separar o conjunto de validação do conjunto de treino temporário
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

num_classes = y_train.shape[1]

# Augmentation
train_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,          # rotação maior
    zoom_range=0.3,              # zoom mais forte
    width_shift_range=0.2,       # deslocamento horizontal maior
    height_shift_range=0.2,      # deslocamento vertical maior
    shear_range=20,              # distorções
    brightness_range=[0.7, 1.3], # brilho variável
    horizontal_flip=True         # flip horizontal
)
val_aug = ImageDataGenerator(preprocessing_function=preprocess_input)
test_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Datasets
batch_size = 32

train_dataset = train_aug.flow(X_train, y_train, batch_size=batch_size)
val_dataset = val_aug.flow(X_val, y_val, batch_size=batch_size)
test_dataset = test_aug.flow(X_test, y_test, batch_size=batch_size, shuffle=False)

Classes encontradas: ['chordata_balaenidae', 'chordata_ardeidae', 'chordata_pleuronectidae', 'chordata_goodeidae', 'chordata_plethodontidae', 'chordata_labridae', 'cnidaria_agariciidae', 'mollusca_cardiidae', 'chordata_cervidae', 'arthropoda_papilionidae', 'chordata_dasypodidae', 'chordata_turdidae', 'chordata_recurvirostridae', 'chordata_tropiduridae', 'chordata_vombatidae', 'cnidaria_dendrophylliidae', 'chordata_carettochelyidae', 'chordata_hylobatidae', 'chordata_soricidae', 'cnidaria_faviidae', 'chordata_balaenicipitidae', 'chordata_strigopidae', 'chordata_gliridae', 'chordata_daubentoniidae', 'chordata_paradisaeidae', 'arthropoda_pseudophasmatidae', 'chordata_diomedeidae', 'chordata_bovidae', 'chordata_parulidae', 'chordata_laridae', 'chordata_vireonidae', 'chordata_cebidae', 'chordata_callitrichidae', 'arthropoda_formicidae', 'chordata_diplodactylidae', 'cnidaria_fungiidae', 'chordata_vespertilionidae', 'chordata_ambystomatidae', 'chordata_scolopacidae', 'chordata_equidae', 'chor

In [6]:
# Converter y_train de one-hot para label (índice)
y_train_labels = np.argmax(y_train, axis=1)

In [7]:
from sklearn.utils.class_weight import compute_class_weight

# Calcular os pesos
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

# Transformar para dict (como o Keras espera)
class_weights = dict(enumerate(class_weights))

print("Class weights:", class_weights)

Class weights: {0: np.float64(1.9771727172717273), 1: np.float64(0.9885863586358636), 2: np.float64(1.9771727172717273), 3: np.float64(1.9771727172717273), 4: np.float64(0.21968585747463634), 5: np.float64(0.9885863586358636), 6: np.float64(0.4942931793179318), 7: np.float64(0.9885863586358636), 8: np.float64(1.9771727172717273), 9: np.float64(1.9771727172717273), 10: np.float64(1.9771727172717273), 11: np.float64(0.9885863586358636), 12: np.float64(1.9771727172717273), 13: np.float64(1.9771727172717273), 14: np.float64(1.9771727172717273), 15: np.float64(1.9771727172717273), 16: np.float64(1.9771727172717273), 17: np.float64(0.659057572423909), 18: np.float64(1.9771727172717273), 19: np.float64(0.9885863586358636), 20: np.float64(1.9771727172717273), 21: np.float64(1.9771727172717273), 22: np.float64(1.9771727172717273), 23: np.float64(1.9771727172717273), 24: np.float64(1.9771727172717273), 25: np.float64(1.9771727172717273), 26: np.float64(0.3295287862119545), 27: np.float64(0.24714

## Model

In [8]:
base_model = EfficientNetB3(include_top=False, weights="imagenet", input_tensor=Input(shape=(299, 299, 3)))
base_model.trainable = False 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2,
    class_weight=class_weights  # <-- aqui!
)

# Fine-tuning
base_model.trainable = True
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stop, reduce_lr],
    verbose=2,
    class_weight=class_weights  # <-- aqui também!
)

/Users/joaosantos/.pyenv/versions/3.10.12/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


/Users/joaosantos/.pyenv/versions/3.10.12/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 299, 299, 3))
  warnings.warn(msg)



Epoch 1: val_loss improved from inf to 5.01572, saving model to melhor_modelo.keras
225/225 - 303s - 1s/step - AUC: 0.5696 - accuracy: 0.0134 - loss: 5.3409 - val_AUC: 0.7214 - val_accuracy: 0.1026 - val_loss: 5.0157 - learning_rate: 1.0000e-04
Epoch 2/30

Epoch 2: val_loss improved from 5.01572 to 4.72052, saving model to melhor_modelo.keras
225/225 - 294s - 1s/step - AUC: 0.6981 - accuracy: 0.0558 - loss: 4.9949 - val_AUC: 0.8345 - val_accuracy: 0.2520 - val_loss: 4.7205 - learning_rate: 1.0000e-04
Epoch 3/30

Epoch 3: val_loss improved from 4.72052 to 4.33322, saving model to melhor_modelo.keras
225/225 - 293s - 1s/step - AUC: 0.7894 - accuracy: 0.1242 - loss: 4.6460 - val_AUC: 0.9094 - val_accuracy: 0.3567 - val_loss: 4.3332 - learning_rate: 1.0000e-04
Epoch 4/30

Epoch 4: val_loss improved from 4.33322 to 3.88546, saving model to melhor_modelo.keras
225/225 - 1149s - 5s/step - AUC: 0.8534 - accuracy: 0.1991 - loss: 4.2098 - val_AUC: 0.9427 - val_accuracy: 0.4043 - val_loss: 3.885

Check some information about scores: 

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Carregar o modelo salvo
modelo_final = load_model("melhor_modelo.keras")

# Previsões
y_pred_probs = modelo_final.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Matriz de confusão
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap="Blues")
plt.title("Matriz de Confusão")
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.show()

------------------------------

### Tentativa errada de tentar aplicar deeplab3 

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import DenseNet201

# Caminho para o diretório com as imagens
base_dir = "/Users/joaosantos/Documents/Mestrado Joao/2 semestre/Deep Learning/rare_speciee"

def load_deeplab_model(input_size=(512, 512)):
    """Carrega o modelo DeepLabV3+ pré-treinado do TensorFlow (usando DenseNet201 como substituto)."""
    deeplab_model = tf.keras.applications.DenseNet201(input_shape=input_size + (3,), include_top=False, weights='imagenet')
    return deeplab_model

def segment_image(model, image_path, target_size=(512, 512)):
    """Aplica o modelo DeepLabV3+ para segmentação de uma imagem."""
    try:
        img = image.load_img(image_path, target_size=target_size)
    except (IOError, UnidentifiedImageError) as e:
        print(f"Erro ao carregar a imagem {image_path}: {e}")
        return None  # Retorna None para indicar que a imagem não foi carregada corretamente
    
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normaliza a imagem
    
    # Realiza a segmentação
    predictions = model.predict(img_array)
    
    # Convertendo a previsão em uma máscara binária
    prediction_mask = np.argmax(predictions[0], axis=-1)
    return prediction_mask

def apply_mask(image_path, mask, target_size=(64, 64)):
    """Aplica a máscara segmentada à imagem original."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, target_size)  # Redimensiona para o tamanho desejado
    
    # Verificar se a máscara não está vazia
    if mask is None or mask.size == 0:
        raise ValueError(f"A máscara gerada está vazia para a imagem {image_path}")
    
    mask_resized = cv2.resize(mask.astype(np.uint8), target_size)  # Redimensiona a máscara para o mesmo tamanho da imagem
    
    # Aplica a máscara (multiplicação elemento a elemento)
    masked_image = np.multiply(img_resized, mask_resized[..., None])  # Canal RGB (três canais)
    
    return masked_image

def load_images_with_segmentation(base_dir, model, image_size=(64, 64), is_train=True):
    """Carrega imagens e aplica a segmentação (DeepLabV3+) nas imagens de treinamento."""
    X_data = []
    y_data = []
    
    # Iterar pelas pastas (famílias) no diretório base
    labels = os.listdir(base_dir)
    labels = [label for label in labels if os.path.isdir(os.path.join(base_dir, label)) and not label.startswith('.')]
    
    label_map = {label: idx for idx, label in enumerate(labels)}

    for label in labels:
        class_dir = os.path.join(base_dir, label)
        
        # Ignorar a pasta 'processed', mas carregar as imagens da pasta da classe
        if "processed" in os.listdir(class_dir):
            print(f"Ignorando a pasta 'processed' na pasta {label}")
        
        # Iterar sobre as imagens na pasta da classe (mesmo com a pasta "processed" presente)
        for img_name in os.listdir(class_dir):
            # Ignorar a pasta 'processed'
            if img_name == "processed":
                continue

            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Erro ao carregar a imagem: {img_path}")
                continue  # Ignora a imagem se não for carregada corretamente
            else:
                print(f"Imagem carregada com sucesso: {img_path}")
                
            # Aplicar segmentação para imagens de treino
            if is_train:
                mask = segment_image(model, img_path)
                if mask is not None:
                    img_data = apply_mask(img_path, mask, target_size=image_size)
                else:
                    continue  # Pula a imagem se não foi carregada corretamente
            else:
                # Para dados de teste, carregamos a imagem original
                img_data = cv2.resize(img, image_size)  # Redimensiona a imagem

            X_data.append(img_data)
            y_data.append(label_map[label])

    # Verificar se algum dado foi carregado
    if len(X_data) == 0 or len(y_data) == 0:
        print("Nenhuma imagem foi carregada. Verifique se o diretório está correto e se as imagens existem.")
    
    # Converter para numpy arrays e normalizar
    X_data = np.array(X_data, dtype="float32") / 255.0
    y_data = np.array(y_data)
    
    # One-hot encoding dos rótulos
    num_classes = len(labels)
    y_data = to_categorical(y_data, num_classes=num_classes)
    
    return X_data, y_data

# Carregar o modelo DeepLabV3+ diretamente
deeplab_model = load_deeplab_model()

# Carregar os dados de treinamento e teste
X_data, y_data = load_images_with_segmentation(base_dir, deeplab_model, image_size=(64, 64), is_train=True)

# Dividir dados em treinamento e teste (sem segmentação para os dados de teste)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

# Visualizar as dimensões dos dados
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Criar o modelo de classificação
num_classes = y_train.shape[1]  # Número de classes

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3),
           kernel_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Compilar o modelo
optimizer = Adam(learning_rate=0.00005)
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy", "AUC"]
)

# Callbacks
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("melhor_modelo.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Treinamento
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

history = model.fit(
    train_dataset,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2
)